## Update Source JSONs

### gets raw jsons from unhcr

In [ ]:
import json, requests

command = 'wget http://data.unhcr.org/api/stats/mediterranean/countries.json -O json_store/mediterranean_countries.json'
!$command

command = 'wget http://data.unhcr.org/api/stats/origin.json -O json_store/origin_countries.json'
!$command

command = 'wget http://data.unhcr.org/api/countries/list.json -O geo_data/countries_list.json'
!$command                

## gets every combination of monthly arrival data between origin and asylum countries

In [ ]:
#Malta is MLT not MTA, make that change 
with open('json_store/mediterranean_countries.json') as data_file:    
    asylum_countries = json.load(data_file)

with open('json_store/origin_countries.json') as data_file:
    origin_countries = json.load(data_file)

for asyl_country in asylum_countries:
    asylum_str = asyl_country["country_of_asylum"]
    asylum_name = asyl_country["country_of_asylum_en"]
    command = "mkdir json_store/by_country/{0}".format(asylum_name)
    !$command
    command = 'wget http://data.unhcr.org/api/stats/mediterranean/monthly_arrivals_by_country.json?country={0} -O json_store/by_country/{1}/monthly{1}.json'.format(asylum_str, asylum_name)
    !$command
    command = 'wget http://data.unhcr.org/api/stats/mediterranean/monthly_arrivals_by_location.json?country={0} -O json_store/by_country/{1}/monthly_loc{1}.json'.format(asylum_str, asylum_name)
    !$command
    for origin_country in origin_countries:
        origin_str = origin_country["origin"]
        origin_name = origin_country["origin_en"]
        result = requests.get("http://data.unhcr.org/api/stats/mediterranean/origin.json", 
              params={'country_of_asylum': asylum_str, 'country_of_origin': origin_str}).json()
        if result:
            print result
            with open('json_store/by_country/{0}/from{1}.json'.format(asylum_name, origin_name), 'w') as outfile:
                json.dump(result, outfile)

## updates the deaths json, must do manually from link

In [ ]:
import json
deaths = {}
for i in range(2014,2017):
    deaths[i] = {}
    for j in range(1,13):
        deaths[int(i)][int(j)] = 0
        
#from: http://missingmigrants.iom.int/mediterranean

deaths[2014][1] = int(12)
deaths[2014][2] = int(24)
deaths[2014][3] = int(10)
deaths[2014][4] = int(50)
deaths[2014][5] = int(329)
deaths[2014][6] = int(318)
deaths[2014][7] = int(864)
deaths[2014][8] = int(616)
deaths[2014][9] = int(813)
deaths[2014][10] = int(126)
deaths[2014][11] = int(22)
deaths[2014][12] = int(95)
deaths[2015][1] = int(82)
deaths[2015][2] = int(345)
deaths[2015][3] = int(61)
deaths[2015][4] = int(1244)
deaths[2015][5] = int(95)
deaths[2015][6] = int(11)
deaths[2015][7] = int(230)
deaths[2015][8] = int(686)
deaths[2015][9] = int(268)
deaths[2015][10] = int(432)
deaths[2015][11] = int(106)
deaths[2015][12] = int(210)
deaths[2016][1] = int(369)
deaths[2016][2] = int(54)
deaths[2016][3] = int(325)
deaths[2016][4] = int(630)
deaths[2016][5] = int(1138)
deaths[2016][6] = int(389)
deaths[2016][7] = int(129)
deaths[2016][8] = int(0)
deaths[2016][9] = int(0)
deaths[2016][10] = int(0)
deaths[2016][11] = int(0)
deaths[2016][12] = int(0)

print deaths

with open('json_store/deaths.json', 'w') as outfile:
    json.dump(deaths, outfile)

## Process Monthly Data

In [9]:
import json

from os import listdir
from os.path import isfile, join, isdir

countries_root = 'json_store/by_country/'

country_folders = [f for f in listdir(countries_root) if not isdir(f)]
print country_folders

#calculates amount of people who immigrated into a country from 
#a certain place within the month 
def immigrated_in_month_from(total_year, total_from_year, total_month):
    percent_from = total_from_year/float(total_year)
    return int(round(total_month*percent_from, 0))
    


# with open('json_store/monthly_by_country.json') as data_file:    
#     monthly_data = json.load(data_file)

['Greece', 'Spain', 'Italy']


In [10]:
all_data = {}

for country_folder in country_folders:
    country_obj = {}
    print country_folder
    with open('{0}{1}/monthly{1}.json'.format(countries_root, country_folder)) as data_file: 
        data_f = json.load(data_file)
        prevYear = data_f[0]["year"] 
        all_months = {}
        monthly = {}
        yearly = {}
        for data in data_f:
            yearly[data["year"]] = yearly.get(data["year"], 0) + data["value"]
            if data["year"] == prevYear:
                all_months[data["month"]] = data["value"]
            else:
                monthly[prevYear] = all_months
                all_months = {}
                all_months[data["month"]] = data["value"]
            prevYear = data["year"]
        monthly[prevYear] = all_months
        all_months[data["month"]] = data["value"]
        country_obj["yearly"] = yearly
        country_obj["monthly"] = monthly
        
    origin_files = [f for f in listdir(countries_root+country_folder) if 'from' in f]
    origins = {}
    for origin_file in origin_files:
        with open('{0}{1}/{2}'.format(countries_root, country_folder, origin_file)) as data_file:
                data = json.load(data_file)[0]
                origins[data["country_of_origin_en"]] = data["value"]
    country_obj["origins"] = origins
    all_data[country_folder] = country_obj

all_data.pop("Malta", None)

print all_data

with open('json_store/all_data.json', 'w') as outfile:
    json.dump(all_data, outfile)

Greece
Spain
Italy
{'Italy': {'yearly': {2016: 56227, 2014: 170100, 2015: 153842}, 'monthly': {2016: {1: 5273, 2: 3828, 3: 9676, 4: 9149, 5: 19925, 6: 8376}, 2014: {1: 2171, 2: 3335, 3: 5459, 4: 15679, 5: 14599, 6: 22641, 7: 24031, 8: 24774, 9: 26107, 10: 15277, 11: 9295, 12: 6732}, 2015: {1: 3528, 2: 4354, 3: 2283, 4: 16063, 5: 21235, 6: 22891, 7: 23186, 8: 22609, 9: 15922, 10: 8916, 11: 3218, 12: 9637}}, 'origins': {u'Ghana': 995, u'Ethiopia': 1012, u'Egypt': 1815, u'Tunisia': 320, u'Senegal': 2565, u'Somalia': 3450, u'Guinea-Bissau': 254, u'Mali': 2614, u'Cameroon': 777, u'Sierra Leone': 217, u'Benin': 183}}, 'Spain': {'yearly': {2016: 1352, 2014: 4348, 2015: 4408}, 'monthly': {2016: {1: 447, 2: 180, 3: 276, 4: 449, 5: 0, 6: 0}, 2014: {1: 144, 2: 33, 3: 232, 4: 148, 5: 325, 6: 246, 7: 264, 8: 1705, 9: 380, 10: 341, 11: 211, 12: 319}, 2015: {1: 241, 2: 44, 3: 267, 4: 240, 5: 438, 6: 377, 7: 346, 8: 385, 9: 459, 10: 795, 11: 508, 12: 308}}, 'origins': {u'Kenya': 4, u'Cameroon': 191, u

In [11]:
processed_data = {}
with open('json_store/all_data.json') as data_file: 
    data = json.load(data_file)
    proc_country_data = {}
    for country_code in country_folders:
        country_data = data[country_code]
        origin_data = country_data["origins"]
        years_data = {}
        for i in range(0, 3):
            year = unicode(i + 2014)
            year_data = {}
            yearsum = 0;
            if "yearly" in country_data:
                for year_key, year_val in country_data["yearly"].iteritems():
                    yearsum += year_val
                months_data = {}
                for month_key, month_val in sorted(country_data["monthly"][year].iteritems()):
                    origin_data = {}
                    for origin, origin_val in country_data["origins"].iteritems():
                        origin_data[str(origin)] = immigrated_in_month_from(yearsum, origin_val, month_val)
                    months_data[int(month_key)] = origin_data
                years_data[int(year)] = months_data
        proc_country_data[country_code] = years_data
processed_data = proc_country_data

with open('json_store/deaths.json') as data_file:            
    processed_data["deaths"] = json.load(data_file)

print processed_data

with open('json_store/processed_data.json', 'w') as outfile:
    json.dump(processed_data, outfile)

{'deaths': {u'2015': {u'11': 106, u'10': 432, u'12': 210, u'1': 82, u'3': 61, u'2': 345, u'5': 95, u'4': 1244, u'7': 230, u'6': 11, u'9': 268, u'8': 686}, u'2014': {u'11': 22, u'10': 126, u'12': 95, u'1': 12, u'3': 10, u'2': 24, u'5': 329, u'4': 50, u'7': 864, u'6': 318, u'9': 813, u'8': 616}, u'2016': {u'11': 0, u'10': 0, u'12': 0, u'1': 369, u'3': 325, u'2': 54, u'5': 1138, u'4': 630, u'7': 129, u'6': 389, u'9': 0, u'8': 0}}, 'Italy': {2016: {1: {'Ghana': 14, 'Tunisia': 4, 'Egypt': 25, 'Ethiopia': 14, 'Senegal': 36, 'Somalia': 48, 'Guinea-Bissau': 4, 'Mali': 36, 'Cameroon': 11, 'Sierra Leone': 3, 'Benin': 3}, 2: {'Ghana': 10, 'Tunisia': 3, 'Egypt': 18, 'Ethiopia': 10, 'Senegal': 26, 'Somalia': 35, 'Guinea-Bissau': 3, 'Mali': 26, 'Cameroon': 8, 'Sierra Leone': 2, 'Benin': 2}, 3: {'Ghana': 25, 'Tunisia': 8, 'Egypt': 46, 'Ethiopia': 26, 'Senegal': 65, 'Somalia': 88, 'Guinea-Bissau': 6, 'Mali': 67, 'Cameroon': 20, 'Sierra Leone': 6, 'Benin': 5}, 4: {'Ghana': 24, 'Tunisia': 8, 'Egypt': 44

## Process Geo Data

In [7]:
import json, csv

with open('geo_data/country_list.json') as data_file:    
    countries = json.load(data_file)

with open('geo_data/country_lat_long.csv', 'r') as csvfile:
    country_lat_lon = {}
    fieldnames = ['name', 'lat', 'lon']
    reader = csv.DictReader(csvfile, fieldnames=fieldnames)
    for row in reader:
        try:
            lat_lon = {}
            lat_lon["lat"] = float(row["lat"])
            lat_lon["lon"] = float(row["lon"])
            country_lat_lon[str(row["name"])] = lat_lon
        except ValueError:
            print "Invalid: {0}, {1}, {2}".format(row["name"],row["lat"],row["lon"])

def get_country_code(name):
    for country in countries:
        if name.lower() == country["name_en"].lower():
            return country["country_code"]
    return None

def get_country_name(code):
    for country in countries:
        if code.lower() == country["country_code"].lower():
            return country["name_en"]
    return None

def get_all_country_codes():
    return [country["country_code"] for country in countries]

def get_lat_lon_by_name(name):
    return country_lat_lon[name]

def get_lat_lon_by_code(code):
    return get_lat_lon_by_name(get_country_name(code))

Invalid: U.S. Minor Outlying Islands, , 


## defines lat long for each country and paths

In [507]:
geo_data = {}
path_geo = {}
with open('json_store/all_data.json') as data_file:
    data = json.load(data_file)
    asyl_geo = {}
    origin_geo = {}
    for key in data:
        asyl_geo[str(key)] = get_lat_lon_by_name(key)
        for or_key in data[key]["origins"]:
            origin_geo[str(or_key)] = get_lat_lon_by_name(or_key)
            if not (key in path_geo): 
                path_geo[key] = {}
            if not (or_key in path_geo[key]):
                path_geo[key][or_key] = []
geo_data['asylum_geo'] = asyl_geo
geo_data['origin_geo'] = origin_geo

path_geo = {}
with open("geo_data/path_by_country.json") as data_file:
    data = json.load(data_file);
    for or_key in data:        
        path_geo[str(or_key)] = {}
        for asyl_key in data[or_key]:
            path_geo[or_key][str(asyl_key)] = [get_lat_lon_by_name(str(country)) for country in data[or_key][asyl_key]]

geo_data["path_geo"] = path_geo

print geo_data

with open('geo_data/geo_data.json', 'w') as outfile:
    json.dump(geo_data, outfile)

{'origin_geo': {'Ghana': {'lat': 7.946527, 'lon': -1.023194}, 'Afghanistan': {'lat': 33.93911, 'lon': 67.709953}, 'Uganda': {'lat': 1.373333, 'lon': 32.290275}, 'Tunisia': {'lat': 33.886917, 'lon': 9.537499}, 'Egypt': {'lat': 26.820553, 'lon': 30.802498}, 'Syrian Arab Rep.': {'lat': 34.802075, 'lon': 38.996815}, 'Ethiopia': {'lat': 9.145, 'lon': 40.489673}, 'Senegal': {'lat': 14.497401, 'lon': -14.452362}, 'Pakistan': {'lat': 30.375321, 'lon': 69.345116}, 'Somalia': {'lat': 5.152149, 'lon': 46.199616}, 'Guinea-Bissau': {'lat': 11.803749, 'lon': -15.180413}, 'Mali': {'lat': 17.570692, 'lon': -3.996166}, 'Kenya': {'lat': -0.023559, 'lon': 37.906193}, 'Iraq': {'lat': 33.223191, 'lon': 43.679291}, 'Cameroon': {'lat': 7.369722, 'lon': 12.354722}, 'Sierra Leone': {'lat': 8.460555, 'lon': -11.779889}, 'Benin': {'lat': 9.30769, 'lon': 2.315834}}, 'asylum_geo': {'Italy': {'lat': 41.87194, 'lon': 12.56738}, 'Spain': {'lat': 40.463667, 'lon': -3.74922}, 'Greece': {'lat': 39.074208, 'lon': 21.8243

## Generate Particle Binary File

In [539]:
import array, datetime, math, random

def get_path_through_points(points):
    curves = []
    currentCursor = points[0]
    i = 0
    for i in range(1, len(points) - 2):
        cntrlPoint = get_control_point(points[i], points[i + 1])
        curves.append([currentCursor, points[i], cntrlPoint]);
        currentCursor = cntrlPoint;
    i += 1
    curves.append([currentCursor, points[i], points[i + 1]]);
    return curves
        
def get_control_point(start, end):
    return [(start[0] + end[0]) / 2, (start[1] + end[1]) / 2]

def bezier_curve_length(curve):
    start = {
        'x': curve[0][0], 
        'y': curve[0][1]
    }
    control = {
        'x': curve[1][0], 
        'y': curve[1][1]
    }
    end = {
        'x': curve[2][0], 
        'y': curve[2][1]
    }
    
    a = {'x':start['x'] - 2 * control['x'] + end['x'],
         'y':start['y'] - 2 * control['y'] + end['y']}
    
    b = {'x': 2 * control['x'] - 2 * start['x'],
         'y': 2 * control['y'] - 2 * start['y']}
    
    A = 4 * (a['x'] * a['x'] + a['y'] * a['y'])
    B = 4 * (a['x'] * b['x'] + a['y'] * b['y'])
    C = b['x'] * b['x'] + b['y'] * b['y']
    
    Sabc = 2 * math.sqrt(A + B + C);
    A_2 = math.sqrt(A);
    A_32 = 2 * A * A_2;
    C_2 = 2 * math.sqrt(C)
    BA = B / A_2
    
    return (A_32 * Sabc + A_2 * B * (Sabc - C_2) +
        (4 * C * A - B * B) * math.log((2 * A_2 + BA + Sabc) / (BA + C_2))) / (4 * A_32);


def LonLatToPixelXY(lon, lat):
    x = (lon + 180) / 360 * 256
    y = ((1 - math.log(math.tan(lat * math.pi / 180) + 1 / math.cos(lat * math.pi / 180)) / math.pi) / 2 * math.pow(2, 0)) * 256
    return [x, y]

def YearMonthDayHourToEpoch(year, month, day, hour):
    return (datetime.datetime(int(year), int(month), int(day), int(hour)) - datetime.datetime(1970, 1, 1, 0)).total_seconds()

def random_day_hour(month):
    hour = random.randint(0,23)
    if month == 2:
        day = random.randint(1, 28)
    elif month == 4 or month == 6 or month == 9 or month == 11:
        day = random.randint(1, 30)
    else:
        day = random.randint(1,31)
    return (day, hour)

def get_points_perpindicular(x1, y1, x2, y2, offset):
    k = offset/(math.sqrt(((y1 - y2)**2) + ((x2 - x1)**2))) 
    return ([x1 + (y1-y2)*k, y1 + (x2-x1)*k], [x2 + (y1-y2)*k, y2 + (x2-x1)*k])

def get_midpoint(x1, y1, x2, y2):
    return ((x1+x2)/2, (y1+y2)/2)

def get_slope(x1, y1, x2, y2):
    return (y2-y1)/(x2-x1)

def get_intersection(x1,y1,x2,y2,x3,y3,x4,y4):
    m1 = get_slope(x1,y1,x2,y2)
    m2 = get_slope(x3,y3,x4,y4)
    x = (m1*x2-y2-m2*x3+y3)/(m1-m2)
    y = m1*x-m1*x2+y2
    return [x,y]
    
def quad_to_cubic(startPoint, controlPoint, endPoint):
    cP1 = [startPoint[i] + (2/3.)*(controlPoint[i]-startPoint[i]) for i in range(2)]
    cP2 = [endPoint[i] + (2/3.)*(controlPoint[i]-endPoint[i]) for i in range(2)]
    return (startPoint, cP1, cP2, endPoint)

def cubic_to_quad(startPoint, controlPoint1, controlPoint2, endPoint):
    cP = [-0.25*startPoint[i] + .75*controlPoint1[i] + .75*controlPoint2[i] -0.25*endPoint[i] for i in range(2)]
    return (startPoint, cP, endPoint)

def offsetCubic(startPoint, controlPoint1, controlPoint2, endPoint, offset):
    offStart, point1 = get_points_perpindicular(startPoint[0], startPoint[1], controlPoint1[0], controlPoint1[1], offset)
    point4,offEnd = get_points_perpindicular(controlPoint2[0], controlPoint2[1], endPoint[0], endPoint[1], offset)
    point2, point3 = get_points_perpindicular(controlPoint1[0], controlPoint1[1], controlPoint2[0], controlPoint2[1], offset)
    offCP1 = get_intersection(offStart[0], offStart[1], point1[0], point1[1], point2[0], point2[1], point3[0], point3[1])
    offCP2 = get_intersection(point2[0], point2[1], point3[0], point3[1], point4[0], point4[1], offEnd[0], offEnd[1])
    return (offStart, offCP1, offCP2, offEnd)

class Particle:
    'A particle within a given Bezier curve'
    def __init__(self, travelTime, startPoint, controlPoint, endPoint):
        self.travelTime = travelTime
        self.startPoint = startPoint
        self.controlPoint = controlPoint
        self.endPoint = endPoint
        
    def addToBuff(self, startTime, offset, isStart, isEnd, buff):
        cubic = quad_to_cubic(self.startPoint, self.controlPoint, self.endPoint)
        offStart, offCP1, offCP2, offEnd = offsetCubic(cubic[0], cubic[1], cubic[2], cubic[3], offset)
        start, control, end = cubic_to_quad(offStart, offCP1, offCP2, offEnd)
        buff += start if not isStart else self.startPoint 
        buff += end if not isEnd else self.endPoint
        buff += control
        buff.append(startTime)
        buff.append(self.travelTime)
    
    def __str__(self):
        return "{0},{1},{2},{3}".format(self.startPoint, self.controlPoint, self.endPoint, self.travelTime)
        
class ParticleGenerator:
    'Generates particles for a given month to add to the buffer'
    def __init__(self, year, month, points, amount, time_to_complete, buff):
        self.startTime = YearMonthDayHourToEpoch(year, month, 1, 0)
        self.month = month
        self.year = year
        self.points = points
        self.amount = amount
        self.time_to_complete = time_to_complete
        self.buff = buff
    
    def generate_particles(self):
        curves = get_path_through_points(self.points)
        lengths = [bezier_curve_length(curve) for curve in curves]
        length_sum = sum(lengths)
        time_lengths = [self.time_to_complete*(length/length_sum) for length in lengths]
        particleset = [Particle(time_lengths[i], curves[i][0], curves[i][1], curves[i][2]) for i in range(len(curves))]
        offsetChoices = [random.uniform(-1,1)*.075*math.sqrt(self.amount) for i in range(int(math.sqrt(self.amount)))]
        for i in range(self.amount):
            if i % 20 == 0:
                offset = 0
            else:
                offset = random.choice(offsetChoices);
            
            time_offset = self.time_to_complete if not self.year==2014 or not self.month==1 else -1 
            day, hour = random_day_hour(self.month)
            time = YearMonthDayHourToEpoch(self.year, self.month, day, hour) - time_offset
                
            starter = True
            ender = len(particleset) == 1
            particleset[0].addToBuff(time, offset, starter, ender, buff)
            starter = False
            for j in range(1,len(particleset)):
                ender = (j == len(particleset) - 1)
                time += time_lengths[j-1]
                particleset[j].addToBuff(time, offset, starter, ender, buff)
                
# cubic = quad_to_cubic([1,1],[2,3],[5,5])
# print cubic
# offsetCubic = offset(cubic[0], cubic[1], cubic[2], cubic[3], 1)
# print offsetCubic
# quad = cubic_to_quad(offsetCubic[0], offsetCubic[1], offsetCubic[2], offsetCubic[3])
# print quad

In [540]:
try:
    processed_data
except NameError:
    with open('json_store/processed_data.json') as data_file:
        processed_data = json.load(data_file)
processed_data.pop("deaths", None)
try:
    geo_data
except NameError:
    with open('geo_data/geo_data.json') as data_file:
        geo_data = json.load(data_file)

one_minute = 60
one_hour= one_minute * 60
one_day = one_hour * 24
two_days = 2 * one_day
three_days = 3 * one_day
five_days = 5 * one_day
six_days = 6 * one_day
ten_days = 2 * five_days
twenty_five_days = 5 * five_days
        
total_span = ten_days
        
buff = []

for asyl_country in processed_data:
    yearly = processed_data[asyl_country]
    for year in yearly:
        monthly = yearly[year]
        for month in monthly:
            for origin_country in monthly[month]:
                amount = monthly[month][origin_country]
                path = geo_data['path_geo'][origin_country][asyl_country]
                path_points = [LonLatToPixelXY(lat_lon['lon'], lat_lon['lat']) for lat_lon in path]
                ParticleGenerator(year, month, path_points, amount,total_span, buff).generate_particles()

# origin = 'Syrian Arab Rep.'
# asylum = 'Greece'

# lat_lons = [LonLatToPixelXY(lat_lon['lon'], lat_lon['lat']) for lat_lon in geo_data['path_geo'][origin][asylum]] 
# amount = processed_data[asylum ][2014][1][origin]
# ParticleGenerator(2014,1,lat_lons, amount, total_span, buff).generate_particles()

# origin = 'Mali'
# asylum = 'Italy'

# lat_lons = [LonLatToPixelXY(lat_lon['lon'], lat_lon['lat']) for lat_lon in geo_data['path_geo'][origin][asylum]] 
# amount = processed_data[asylum ][2014][1][origin]
# ParticleGenerator(2014,1,lat_lons, amount, total_span, buff).generate_particles()

array.array('f', buff).tofile(open("./particles.bin", 'w'))
                

In [383]:
math.pow(2,3)


8.0

In [524]:
math.max(1,2)

AttributeError: 'module' object has no attribute 'max'